In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

'''
This script trains the cell segmentation U-Net

@author: jblugagne
Small augmentations: georgeoshardo
'''
# pip install "scikit_image==0.16.2"
import sys
sys.path.insert(1, '../SyMBac/')

from SyMBac.external.DeLTA.model import unet_seg
from SyMBac.external.DeLTA.data import trainGenerator_seg
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from tqdm.auto import tqdm
from natsort import natsorted

2023-05-10 01:39:09.218738: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
from glob import glob

In [3]:
data_dirs = natsorted(glob("data/*"))

In [4]:
data_dirs

['data/1.5_3', 'data/2_4', 'data/3_6', 'data/4_8', 'data/5_10', 'data/6_12']

In [5]:
pbar = tqdm(data_dirs)
for data_dir in pbar:
    pbar.set_description("Processing %s" % data_dir)
    DeLTA_data =  f"{data_dir}/DeLTA/"
    callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=1)

    training_set = DeLTA_data
    model_file = DeLTA_data + 'models/test_new_saved-model-{epoch:02d}-{loss:.4f}.hdf5' #'/unet.hdf5'

    # Parameters:
    target_size = (832,832)
    input_size = target_size + (1,)
    batch_size = 4
    epochs = 100 #25
    steps_per_epoch = 40  #25

    #Data generator:
    data_gen_args = dict(
                        rotation = 0.5,
                        shiftX=.08,
                        shiftY=.08,
                        zoom=.17,
                        horizontal_flip=True,
                        histogram_voodoo=True,
                        illumination_voodoo=True)

    myGene = trainGenerator_seg(batch_size,
                               training_set + 'preprocessed/CROPPED_FILTERED/',
                               training_set + 'preprocessed/CROPPED_MASKS/',
                               training_set + 'preprocessed/WEIGHTMAPS/',
                               augment_params = data_gen_args,
                               target_size = target_size)


    # Define model:
    model = unet_seg(input_size = input_size)
    model.summary()
    model_checkpoint = ModelCheckpoint(model_file, monitor='loss',verbose=1, save_best_only=False, callback=callback)

    #filepath = "saved-model-{epoch:02d}-{loss:.2f}.hdf5"
    # Train it:
    model.fit_generator(myGene,steps_per_epoch=steps_per_epoch,epochs=epochs,callbacks=[model_checkpoint])

  0%|          | 0/6 [00:00<?, ?it/s]

2023-05-10 01:39:09.964624: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-05-10 01:39:09.967424: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-10 01:39:09.967500: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA Graphics Device computeCapability: 8.9
coreClock: 2.55GHz coreCount: 128 deviceMemorySize: 23.69GiB deviceMemoryBandwidth: 938.86GiB/s
2023-05-10 01:39:09.967507: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-05-10 01:39:09.979987: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-05-10 01:39:09.980024: I tensorflow/stream_executor/pla

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
true_input (InputLayer)         [(None, 832, 832, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 832, 832, 64) 640         true_input[0][0]                 
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 832, 832, 64) 36928       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 416, 416, 64) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

2023-05-10 01:39:10.977302: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2023-05-10 01:39:10.977512: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2995200000 Hz


Epoch 1/100


2023-05-10 01:39:11.435372: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2023-05-10 01:39:12.117082: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8801
2023-05-10 01:39:12.467890: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-05-10 01:39:12.467910: W tensorflow/stream_executor/gpu/asm_compiler.cc:56] Couldn't invoke ptxas --version
2023-05-10 01:39:12.468357: E tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-05-10 01:39:12.468397: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2023-05-10 01:39:12.493555: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully ope

40/40 [==============================] - 21s 373ms/step - loss: 0.3575 - unstack_acc: 0.9020

Epoch 00001: saving model to data/1.5_3/DeLTA/models/test_new_saved-model-01-0.3575.hdf5
Epoch 2/100
40/40 [==============================] - 15s 375ms/step - loss: 0.1464 - unstack_acc: 0.9519

Epoch 00002: saving model to data/1.5_3/DeLTA/models/test_new_saved-model-02-0.1464.hdf5
Epoch 3/100
40/40 [==============================] - 15s 376ms/step - loss: 0.1131 - unstack_acc: 0.9612

Epoch 00003: saving model to data/1.5_3/DeLTA/models/test_new_saved-model-03-0.1131.hdf5
Epoch 4/100
40/40 [==============================] - 15s 376ms/step - loss: 0.1095 - unstack_acc: 0.9625

Epoch 00004: saving model to data/1.5_3/DeLTA/models/test_new_saved-model-04-0.1095.hdf5
Epoch 5/100
40/40 [==============================] - 15s 376ms/step - loss: 0.0965 - unstack_acc: 0.9664

Epoch 00005: saving model to data/1.5_3/DeLTA/models/test_new_saved-model-05-0.0965.hdf5
Epoch 6/100
40/40 [==================

KeyboardInterrupt: 

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from SyMBac.external.DeLTA.model import unet_seg
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob
from tqdm.notebook import tqdm
from skimage.transform import resize
from csbdeep.utils import normalize
#from stardist import fill_label_holes, random_label_cmap
#lbl_cmap = random_label_cmap()
from natsort import natsorted

from skimage.filters import threshold_otsu

DeLTA_data =  "data/1.5_3/DeLTA/"
training_set = DeLTA_data
target_size = (832,832)
input_size = target_size + (1,)
process_size = 4096
# Files:

models = natsorted(glob(DeLTA_data + '/models/*'))
model_file = DeLTA_data + "/models/test_new_saved-model-20.hdf5"
model = unet_seg(input_size = input_size)
model.load_weights(model_file)

from skimage.morphology import erosion
a = Image.open("data/6_12//test_data/0020_masks.png")
a = erosion((np.pad(np.array(a),16)>0))*1

data = Image.open("data/6_12//test_data/0020.png")
data = [np.pad(np.array(data),16)]
X = []
for x in range(len(data)):
    X.append(data[x])

requires_resize = True
resize_shape = (832,832) # (320,64) (512,128) (256,64)
if requires_resize:
    X = [resize(x,resize_shape, order=1, mode='reflect', cval=0, clip=True, preserve_range=True, anti_aliasing=None, anti_aliasing_sigma=None) for x in tqdm(X)]

n_channel = 1 if X[0].ndim == 2 else X[0].shape[-1]
axis_norm = (0,1)   # normalize channels independently
# axis_norm = (0,1,2) # normalize channels jointly
all_labels = []
all_imgs = []
for x in tqdm(range(len(X))):
    img = normalize(X[x], 1,99.8, axis=(0,1))
    masks = model.predict(img.reshape(1,832,832,1)).reshape(832,832)
    all_labels.append(masks)
    all_imgs.append(img)
    
mask = all_labels[-1]
plt.figure(figsize=(10,10))
plt.imshow((mask > threshold_otsu(mask)) * 1 - a)